<a href="https://colab.research.google.com/github/parmarsuraj99/10DaysofMLChallenge/blob/master/day7/Day_7_10DaysofMLChallenge.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json
!ls ~/.kaggle

In [0]:
!kaggle datasets download -d nitin194/twitter-sentiment-analysis

In [0]:
!unzip /content/twitter-sentiment-analysis.zip -d data/

In [0]:
import os
import gc
import re
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import wordcloud

In [0]:
train = pd.read_csv("/content/data/train_E6oV3lV.csv")
test = pd.read_csv("/content/data/test_tweets_anuFYb8.csv")

In [0]:
train.head()

In [0]:
train.shape, test.shape

In [0]:
test.head()

So many '#'s of tweets. Can be useful. Lets see some positive and negative tweets

In [0]:
train[train['label']==0].head()

In [0]:
train[train['label']==1].head()

In [0]:
train.label.value_counts()

In [0]:
from wordcloud import WordCloud, STOPWORDS 

Good to see that there is more positivity here! Let's see them from a higher(Cloud) perspective

In [0]:
comment_words = ' '
stopwords = set(STOPWORDS) 

In [0]:
for val in train.tweet[train.label==0][:10]:      
    val = str(val) 
    tokens = val.split()       
    for i in range(len(tokens)): 
        tokens[i] = tokens[i].lower() 
    for words in tokens: 
        comment_words = comment_words + words + ' '

wordcloud = WordCloud(width = 800, height = 800, 
                background_color ='white', 
                stopwords = stopwords, 
                min_font_size = 10).generate(comment_words) 
plt.figure(figsize = (8, 8), facecolor = None) 
plt.imshow(wordcloud) 
plt.axis("off") 
plt.tight_layout(pad = 0) 
  
plt.show() 

We might wan to do some preprocessing here. So It would be better if we combine both sets

In [0]:
full = train.append(test, sort=True)

In [0]:
train.tweet

There are usernames masked with @user. We can remove them. 

In [0]:
def remove_pattern(input_txt, pattern):
    r = re.findall(pattern, input_txt)
    for i in r:
        input_txt = re.sub(i, '', input_txt)
    return input_txt
v_rem = np.vectorize(remove_pattern)

In [0]:
full["clean_tweet"] = v_rem(full["tweet"], "@[\w]*")

In [0]:
full.head()

Can we remove punctuation? Of course!(although, that would mean loss of some important data)

In [0]:
full["clean_tweet"] = full["clean_tweet"].str.replace("[^a-zA-Z#]", " ")

In [0]:
full.head()

In [0]:
r = re.findall("#[\w]*" ,full.clean_tweet.values[3])

In [0]:
from nltk.stem import *
stemmer = PorterStemmer()

In [0]:
tokenized_tweets = full.clean_tweet.apply(lambda x: x.split())

In [0]:
tokenized_tweets = tokenized_tweets.apply(lambda x:[stemmer.stem(plural) for plural in x])

In [0]:
tokenized_tweets = tokenized_tweets.apply(lambda x:" ".join(x))

In [0]:
full["clean_tweet"] = tokenized_tweets

In [0]:
full.head()

Seems good now

In [0]:
all_words = " ".join([tweet for tweet in full.clean_tweet])

In [0]:
wordcloud = WordCloud(width = 800, height = 800, 
                background_color ='white', 
                stopwords = stopwords, 
                min_font_size = 10).generate(all_words) 
plt.figure(figsize = (8, 8), facecolor = None) 
plt.imshow(wordcloud) 
plt.axis("off") 
plt.tight_layout(pad = 0) 
  
plt.show()

In [0]:
non_r_words = " ".join([tweet for tweet in full.clean_tweet[full.label==0]])

In [0]:
wordcloud = WordCloud(width = 800, height = 800, 
                background_color ='white', 
                stopwords = stopwords, 
                min_font_size = 10).generate(non_r_words) 
plt.figure(figsize = (8, 8), facecolor = None) 
plt.imshow(wordcloud) 
plt.axis("off") 
plt.tight_layout(pad = 0) 
  
plt.show()

In [0]:
neg_words = " ".join([tweet for tweet in full.clean_tweet[full.label==1]])

In [0]:
wordcloud = WordCloud(width = 800, height = 800, 
                background_color ='white', 
                stopwords = stopwords, 
                min_font_size = 10).generate(neg_words) 
plt.figure(figsize = (8, 8), facecolor = None) 
plt.imshow(wordcloud) 
plt.axis("off") 
plt.tight_layout(pad = 0) 
  
plt.show()

In [0]:
normal_tags = full[full.label==0]["clean_tweet"].str.findall(r"#[\w]+")
neg_tags = full[full.label==1]["clean_tweet"].str.findall(r"#[\w]+")

In [0]:
normal_tags = sum(normal_tags, [])
neg_tags = sum(neg_tags, [])

In [0]:
normal_max=np.unique(normal_tags, return_counts=True)

In [0]:
normal_max[0][:5]

In [0]:
plt.bar(normal_max[0][:5], normal_max[1][:5])

In [0]:
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer 
import gensim

In [0]:
tfidf_vectorizer = TfidfVectorizer(max_df=0.90, min_df=2, max_features=1000, stop_words='english')
tfidf = tfidf_vectorizer.fit_transform(full['clean_tweet'])
tfidf.shape

In [0]:
from sklearn.model_selection import train_test_split

In [0]:
train.shape[0]

In [0]:
xtrain_tfidf, xvalid_tfidf, ytrain, yvalid = train_test_split(tfidf[:train.shape[0]], train['label'], random_state=42, test_size=0.3)

In [0]:
xtrain_tfidf = xtrain_tfidf[ytrain.index]
xvalid_tfidf = xvalid_tfidf[yvalid.index]

In [0]:
xtrain_tfidf.shape

In [0]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import f1_score

In [0]:
rf = RandomForestClassifier(n_estimators=500, random_state=11, verbose=1, n_jobs=-1)
rf.fit(xtrain_tfidf, ytrain, ) 

In [0]:
prediction = rf.predict(xvalid_tfidf)
print("RFClassifier, f1: ",(f1_score(yvalid, prediction)))

In [0]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier

In [0]:
# Model Training 
print ("Create model ... ")
def build_model():
    model = Sequential()
    model.add(Dense(256, input_dim=1000, activation='relu'))
    model.add(Dropout(0.3))
    model.add(Dense(128, activation='relu'))
    model.add(Dropout(0.3))
    model.add(Dense(64, activation='relu'))
    model.add(Dropout(0.3))
    model.add(Dense(1, activation='sigmoid'))
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    #model.summary()
    return model

print("Compile model ...")
model = build_model()

In [0]:
xtrain_tfidf.toarray().shape

In [0]:
ytrain.shape

In [0]:
hst = model.fit(xtrain_tfidf.toarray(), ytrain, epochs = 10, validation_split=0.2)#validation_data=(xvalid_tfidf.toarray(), yvalid))

In [0]:
history = hst

In [0]:
print(history.history.keys())
# summarize history for accuracy
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()
# summarize history for loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

In [0]:
preds = model.predict(xvalid_tfidf.toarray())

In [0]:
preds = (preds>=0.5).astype(int)

In [0]:
preds.squeeze()

In [0]:
yvalid